# This notebook shows example to load the CHGNet for prediction


In [2]:
from chgnet.model import CHGNet


In [3]:
import numpy as np
from pymatgen.core import Structure

from chgnet.model import CHGNet

# If the above line fails in Google Colab due to numpy version issue,
# please restart the runtime, and the problem will be solved

np.set_printoptions(precision=4, suppress=True)

### Read structure from a json or cif file


In [4]:
try:
    from chgnet import ROOT

    
    structure = Structure.from_file(f"{ROOT}/examples/mp-18767-LiMnO2.cif")
except Exception:
    from urllib.request import urlopen

    url = "https://raw.githubusercontent.com/CederGroupHub/chgnet/main/examples/mp-18767-LiMnO2.cif"
    cif = urlopen(url).read().decode("utf-8")
    structure = Structure.from_str(cif, fmt="cif")

print(structure)

Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.868779   4.634475   5.832507
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP      a    b         c
---  ----  ---  ---  --------
  0  Li+   0.5  0.5  0.37975
  1  Li+   0    0    0.62025
  2  Mn3+  0.5  0.5  0.863252
  3  Mn3+  0    0    0.136747
  4  O2-   0.5  0    0.360824
  5  O2-   0    0.5  0.098514
  6  O2-   0.5  0    0.901486
  7  O2-   0    0.5  0.639176


### Define Model


In [20]:
chgnet = CHGNet.load()
chgnet = CHGNet(device="cuda:1")

# Alternatively you can read your own model
# chgnet = CHGNet.from_file(model_path)

CHGNet initialized with 412,525 parameters


# Predict energy, force, stress, magmom


In [21]:
prediction = chgnet.predict_structure(structure)

for key, unit in [
    ("energy", "eV/atom"),
    ("forces", "eV/A"),
    ("stress", "GPa"),
    ("magmom", "mu_B"),
]:
    print(f"CHGNet-predicted {key} ({unit}):\n{prediction[key[0]]}\n")

CHGNet-predicted energy (eV/atom):
-7.0185370445251465

CHGNet-predicted forces (eV/A):
[[-0.     -0.     -0.0092]
 [ 0.     -0.      0.0092]
 [ 0.     -0.     -0.0049]
 [-0.      0.      0.0049]
 [ 0.     -0.      0.0086]
 [-0.      0.      0.03  ]
 [-0.     -0.     -0.03  ]
 [-0.     -0.     -0.0086]]

CHGNet-predicted stress (GPa):
[[ 0.211  -0.     -0.    ]
 [-0.      0.5808 -0.    ]
 [-0.      0.      0.34  ]]

CHGNet-predicted magmom (mu_B):
[1.0071 1.0071 0.4953 0.4953 0.2748 0.2847 0.2847 0.2748]



# Structure Optimization


In [6]:
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cuda


In [7]:
# Perturb the structure
structure.perturb(0.1)

# Relax the perturbed structure
result = relaxer.relax(structure, verbose=True)
print("Relaxed structure:\n")
print(result["final_structure"])

      Step     Time          Energy          fmax
FIRE:    0 16:43:26      -58.941528        0.092591
Relaxed structure:

Full Formula (Li2 Mn2 O4)
Reduced Formula: LiMnO2
abc   :   2.868779   4.634475   5.832507
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP           a         b         c      magmom
---  ----  --------  --------  --------  ----------
  0  Li+   0.506582  0.495719  0.39624   0.00304922
  1  Li+   0.006582  0.995719  0.636739  0.00304922
  2  Mn3+  0.506582  0.495719  0.879742  3.86942
  3  Mn3+  0.006582  0.995719  0.153237  3.86942
  4  O2-   0.506582  0.995719  0.377314  0.0441356
  5  O2-   0.006582  0.495719  0.115003  0.0386223
  6  O2-   0.506582  0.995719  0.917976  0.0386221
  7  O2-   0.006582  0.495719  0.655665  0.0441358


# Molecular Dynamics


In [ ]:
from chgnet.model.dynamics import MolecularDynamics

md = MolecularDynamics(
    atoms=structure,
    model=chgnet,
    ensemble="nvt",
    temperature=1000,  # in k
    timestep=2,  # in fs
    trajectory="md_out.traj",
    loginterval=10,
)
md.run(500)  # run a 0.1 ps MD simulation

CHGNet will run on cuda
NVT-Berendsen-MD created


# Magmom Visualization


In [ ]:
supercell = structure.make_supercell([2, 2, 2], in_place=False)
print(supercell.composition)

In [ ]:
import random

n_Li = int(supercell.composition["Li+"])
remove_ids = random.sample(list(range(n_Li)), n_Li // 2)

supercell.remove_sites(remove_ids)
print(supercell.composition)

In [ ]:
result = relaxer.relax(supercell)

In [ ]:
import pandas as pd

df_magmom = pd.DataFrame({"Unrelaxed": chgnet.predict_structure(supercell)["m"]})
df_magmom["CHGNet relaxed"] = result["final_structure"].site_properties["magmom"]

In [ ]:
fig = df_magmom.hist(
    nbins=200,
    sharex=True,
    sharey=True,
    backend="plotly",
    barmode="overlay",
    layout={"title": "Magmom distribution"},
    opacity=0.7,
    range_x=[3, 4],
    template="plotly_white",
)
fig.layout.legend.update(title="", x=1, y=1, xanchor="right", yanchor="top")
fig.layout.xaxis.title = "Magnetic moment"
fig.show()